## Comentarios/Passos a serem visitados

* musicas geradas não estão sendo convertidos para MIDI (.abc muito inconsistente?)
* remover metadados potencialmente confundidores dos .abc (título, letra, autor)
* como salvar as musicas geradas de forma automatica?

# Importing Libraries

In [78]:
import tensorflow as tf
import numpy as np
import os
import re
import time
from IPython import display as ipythondisplay
import matplotlib.pyplot as plt
#from tqdm import tqdm #--> buga se importar aqui

from functions.reader import *
from functions.batcher import *
from functions.models import *
from functions.generate import *

In [79]:
#print(len(tf.config.list_physical_devices('GPU'))) # number of GPUs
#assert len(tf.config.list_physical_devices('GPU')) > 0 

# Getting Data

In [80]:
path = 'abcnotation_midi'
files = abc_filenames(datapath=path,count=True)
songs = clean_abc(filenames=files, count=True)
songs_joined = join_songs(songs=songs)

Found 184900 songs in directory
There will be used 184603 songs to train the model


# Mapping Indexes

In [81]:
char2idx, idx2char = abc_mapIndex(songs_joined)
vectorized_songs = vectorize_string(string=songs_joined, 
                                    char2idx=char2idx)
vocab = sorted(set(songs_joined))

# First Test

In [82]:
letsTest(vectorized_songs=vectorized_songs, 
        seq_length=10,
        batch_size=2,
        idx2char=idx2char)


Performing some simple tests to make sure the batch function is working properly
[PASS] test_batch_func_types
[PASS] test_batch_func_shapes
[PASS] test_batch_func_next_step
[PASS] passed all tests!

Example of how it works:

Step   0
  input: [38 23  6 97  4 38 72 74 73 39] (array(['A', '2', ' ', '|', '\n', 'A', 'c', 'e', 'd', 'B'], dtype='<U1'))
  expected output: [23  6 97  4 38 72 74 73 39 72] (array(['2', ' ', '|', '\n', 'A', 'c', 'e', 'd', 'B', 'c'], dtype='<U1'))
Step   1
  input: [42 41 38  6 41 20 41 20 41  6] (array(['E', 'D', 'A', ' ', 'D', '/', 'D', '/', 'D', ' '], dtype='<U1'))
  expected output: [41 38  6 41 20 41 20 41  6 43] (array(['D', 'A', ' ', 'D', '/', 'D', '/', 'D', ' ', 'F'], dtype='<U1'))


# Parameters

In [83]:
# Optimization parameters:
num_training_iterations = 2000  # Increase this to train longer
batch_size = 4  # Experiment between 1 and 64
seq_length = 100  # Experiment between 50 and 500
learning_rate = 5e-3  # Experiment between 1e-5 and 1e-1

# Model parameters: 
vocab_size = len(vocab)
embedding_dim = 256 
rnn_units = 1024  # Experiment between 1 and 2048

# Checkpoint location: 
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "my_ckpt")

# instantiate an optimizer with its learning rate.
# Checkout the tensorflow website for a list of supported optimizers. https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/
# Try using the Adam optimizer to start
optimizer = tf.keras.optimizers.Adam(learning_rate)

# Build Model

In [84]:
from tqdm import tqdm # funciona se importar aqui...

# choose 'cpu' or 'gpu'
device = 'gpu'

# warm up
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)
for iter in tqdm(range(int(2000))):
    # Grab a batch and propagate it through the network
    x_batch, y_batch = get_batch(vectorized_songs, seq_length, batch_size)
    loss = train_step(x_batch, y_batch, model, optimizer, device)

# # instantiate a new model for training using the `build_model` function and the hyperparameters create
# model = build_model(vocab_size, embedding_dim, rnn_units, batch_size)

# # begin training
# history = []
# plotter = PeriodicPlotter(sec=2, xlabel='Iterations', ylabel='Loss')
# if hasattr(tqdm, '_instances'): tqdm._instances.clear() # clear if it exists

# for iter in tqdm(range(int(num_training_iterations))):
#     # Grab a batch and propagate it through the network
#     x_batch, y_batch = get_batch(vectorized_songs, seq_length, batch_size)
#     loss = train_step(x_batch, y_batch, model, optimizer, device)

#     # Update the progress bar
#     history.append(loss.numpy().mean())
#     plotter.plot(history)

#     # Update the model with the changed weights!
#     if iter % 100 == 0:     
#         model.save_weights(checkpoint_prefix)

# Save the trained model and the weights
model.save_weights(checkpoint_prefix)

100%|██████████| 2000/2000 [01:50<00:00, 18.05it/s]


# Generate Text 

In [85]:
# Restore the latest checkpoint
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

# Restore the model weights for the last checkpoint after training
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

# Use the model and the function defined above to generate ABC format text of length 5! 
# As you may notice, ABC files start with "X" - this may be a good start string.

generated_text = generate_text(model, start_string="X:0", generation_length=1000, char2idx=char2idx, idx2char=idx2char)

# Generated songs location: 
new_song_dir = './generated_songs'

for i in enumerate(generated_text):
    print('\nGenerated Song',i)
    print(generated_text)
    #print(song)
    #file_abc = open(new_song_dir+'/'+'generated_song-'+str(i)+'.abc', 'w')
    #file_abc.write(generated_text)
    #file_abc.close()


Generated Song (0, 'X')
X:03053
M:2/4
L:1/8
R:Reel
R:Reel
F: https://www.stumliage.i/slack/f/|e3-{d}c/2^G/2{g}B|  e2 G2|1 eB Ac oe Shreains Mris Pucherinaton # the p.6172
R: 20159
M:2/4
L:1/8
K:C
c2 | (Am)nish Dollectmon", 18 61 (manustlice! (A/2c/2).d/2 [|] 
[V:4] nupt"
M: 6/8
L: 1/8
K: F
D2 | F4G2 | G2c2d2e2 | 
[V:4] c4B2 | 
[V:2] D2D2D2 | 
[V:1] abc2 | A4 | 
[V:2] G,2G,2F,2 | 
[V:4] c'4g4 | 
[V:5] D,>E,4F,2 [|] 
[V:4] D,2D,2>D,2}D,2C,8 [|] 
[V:1] c2A2!breath!D,8- | 
[V:1] AG2F2D2- | 
[V:4] _E,4F,8- [|] 
[V:1] F A D A| F F>F E>E {F}F)| cdcA |1 \
vDA,C^G C4A2 | (F>A E][AD] | d>FD>d | "D"c2 A "D7"d>d | "Gm"gg ge | "D"d3/2f/2f/2f/2 | d/f/d/e/              | f/2/!
e/e/f/e/ | gf ge/2f/2ed2 |]

X:55068
M:9/8
L:1/8
F:Dount Air
D:mordhen Vol.S.
R:sott, onathspey
M:6/8
L:1/8
K:Dmix
FA|\
(F2G2) F2A2|(GE2) (FD) A2B2|(cBAg) (dBB) B2f2|(G>e)2a3e|d2BA (d>te)file).kbucs 2nd hromenes RicA E2 D2|E2G G2 A GA G2|F2 A,,2 Lend in file Martlascdeaw Gile na_4F2_E2 | F2F2 G2z2
c/2{dcd |
DF/2A/2B/2<Fe2d  | 